In [1]:
import pandas as pd
import plotly.express as px

**Podsumowanie**  ✅

- Jak rozkłada się aktywność użytkowników w ramach zadanych obszarów tematycznych?

Na równi temat 'PiS' i 'wybory', niedużo niżej 'konfederacja'. W około połowie temat 'referendum' i najmniej 'koalicja'.
- Czy wydźwięk postów wpływa na aktywność użytkowników?

Możnaby powiedzieć, że tak. Najwięcej aktywności, we wszystkich tematach jest do postów o negatywnych wydźwiękach, trochę mniej o neutralnym i dużo mniej o pozytywnym. Ale wykorzystany model również znalazł o wiele mniej postów pozytywnych od reszty, więc cieżko stwierdzić jakąś prawidłowość.
- Czy tematyka postów (używane słowa) wpływają na popularność postów?

Uzyskałem podział na 3 temty: 'Czyny PiSu', 'Partie', oraz 'Mechanizm wyborów'. Najwięcej postów nawiązywało do tematyki partii, a najmniej do 'Machanizmu wyborów', tak samo rozkładała się aktywność dla tych grup tematycznych.

Napisz końcowe podsumowanie. Spróbuj odpowiedzieć na pytania z początku listy. Wskaż najważniejsze elementy analiz, przedstaw mocne i słabe strony rozwiązań. Zaproponuj inne aspekty, które nie zostały ujęte w zadaniu, mające wpływ na partycypacje użytowników.

---

In [2]:
wybory_df = pd.read_csv('wybory_reddit_zid.csv')

In [6]:
#usunięcie wpisów botów
wybory_df = wybory_df.drop_duplicates(subset='body')

In [7]:
wybory_df['body'].nunique()

4297

In [9]:
wybory_df.loc[wybory_df["author"] == "None", "author"] = "ANON"

In [10]:
wybory_users_count = wybory_df.groupby('author').size().reset_index(name='count')
wybory_users_count = wybory_users_count.sort_values(by='count', ascending=False)

wybory_users_count.head()

,author,count
31,ANON,130
155,Bielin_Clash,54
175,BubsyFanboy,45
871,QzinPL,42
1451,kalarepar,42


# Analiza aktywności w tematach

- Rozkład liczby postów opublikowanych przez użytkowników w zadanych tematach
- Rozkład liczby komentarzy opublikowanych przez użytkowników w zadanych tematach
- Procent wszystkich użytkowników piszących posty w obu tematach
- Procent wszystkich użytkowników piszących komentarze w obu tematach
- Rozkład łącznej aktywności (suma reakcji, komentarzy, udostępnień) dotyczącej postów w zadanych tematach

In [ ]:
df_1 = wybory_df.groupby(['author', 'topic']).size().reset_index(name='count')

In [ ]:
df_1[df_1["author"]=="kalarepar"]

,author,topic,count
2155,kalarepar,koalicja,7
2156,kalarepar,konfederacja,11
2157,kalarepar,pis,10
2158,kalarepar,referendum,4
2159,kalarepar,wybory,7


In [ ]:
#Grupowanie po autorach i posortowanie
wybory_users_count = wybory_df.groupby('author').size().reset_index(name='count')
wybory_users_count = wybory_users_count.sort_values(by='count', ascending=False)
sorted_users = list(wybory_users_count['author'])

top20users_df = df_1[df_1['author'].isin(sorted_users[:20])]

wybory_users_count[wybory_users_count["author"]=="kalarepar"]

,author,count
1327,kalarepar,39


In [ ]:
fig = px.bar(top20users_df, x='topic', y='count', color='author',
              title='Ilość postów użytkownika o danej tematyce')

fig.update_xaxes(title='Tematyka')
fig.update_yaxes(title='Ilość postów')

In [ ]:
df_grouped_topics = wybory_df.groupby('author')['topic'].nunique().reset_index()

count_rows = len(df_grouped_topics[df_grouped_topics['topic'] == 5])
count_rows

58

In [ ]:
names = ['Aktywni', 'Nieaktywni']
nums = [count_rows, len(df_grouped_topics) - count_rows]

fig = px.pie(names=names, values=nums, title='Aktywność użytkowników we wszystkich tematach',
             width=600, height=500)
fig.show()

In [ ]:
count_rows = len(df_grouped_topics[df_grouped_topics['topic'] >= 3])
count_rows

275

In [ ]:
names = ['Aktywni', 'Nieaktywni']
nums = [count_rows, len(df_grouped_topics) - count_rows]

fig = px.pie(names=names, values=nums, title='Aktywność użytkowników w conajmniej 3 tamatach',
             width=600, height=500)
fig.show()

In [ ]:
wybory_df['total_act'] = wybory_df['score'] + wybory_df['num_comments']

In [ ]:
df_total = wybory_df.groupby('topic').sum().reset_index()

<ipython-input-20-6b381d1c2daa>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
fig = px.bar(df_total, x='topic', y='total_act', width=800, height=500,
             title='Całościowa aktywność w tematach')

fig.show()

#Wydźwięk

Sprawdź czy wydźwięk danego postu wpływa na aktywność. Analiza powinna być przeprowadzona osobno dla każdego typu aktywności (komentarz, udostępnienie lub reakcje) występującej w zbiorze danych oraz osobno dla każdego z wybranych tematów.

In [ ]:
!pip install huggingface_hub

In [21]:
!git config --global credential.helper store


In [22]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [ ]:
!pip install transformers

In [ ]:
from transformers import *
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)
for pred in nlp(f"Adam Mickiewicz wielkim polskim {nlp.tokenizer.mask_token} był."):
  print(pred)

In [ ]:
!pip install sentimentpl
!pip install sacremoses
from sentimentpl.models import SentimentPLModel

In [ ]:
model = SentimentPLModel(from_pretrained='latest')

In [29]:
print(model('Pointeresuje ch żeby zabezpieczś zaw').item())

-0.1556425839662552


In [27]:
#CZyszczenie z niepotrzebnych znaków

wybory_df['body'] = wybory_df['body'].str.replace(r'[#&%+:()!,;/.,\[\\1234567890]',
                                                  '', regex=True)
wybory_df['body'] = wybory_df['body'].str.replace(r'[-]',
                                                  ' ', regex=True)
wybory_df['body'] = wybory_df['body'].str.replace(r'[|]',
                                                  ' ', regex=True)

In [26]:
def get_sentiment(text):
    try:
        sentiment = model(text).item()
        return sentiment
    except Exception as e:
      print("error")
      return 'er'  # Replace with 0 for rows with errors

In [ ]:
wybory_df['sentiment_score'] = wybory_df['body'].apply(get_sentiment)

In [34]:
# Dla zbyt długich tekstów okrojenie (tylko 120)

for idx, row in wybory_df.iterrows():
  if row['sentiment_score']=='er':
    score = model(row['body'][:1500]).item()
    wybory_df.loc[idx, 'sentiment_score'] = score

In [37]:
fig = px.histogram(wybory_df, x='sentiment_score',
                   width=800, height=500,
                   title='Histogram sentymentu', nbins=200)
fig.show()

In [43]:
#kodowanie sebtymentu na kategorie

wybory_df['sentiment_score'] = wybory_df['sentiment_score'].astype(float)

wybory_df['sentiment'] = pd.cut(
    wybory_df['sentiment_score'],
    bins=[-1, -0.2, -0.009, 1],
    labels=['negatywny', 'neutralny', 'pozytywny']
)

In [39]:
wybory_df.head()

,body,created,author,num_comments,score,downs,topic,id,sentiment_score,sentiment
0,W krótce wybory a ja sobie zdałam sprawę że ni...,2023-08-31,krowajestes,101,48,0,wybory,1668zix,-0.576105,negatywny
1,Mam takie pytanie Będzie to mój pierwszy udzia...,2023-10-14,DDDOMIK,34,0,0,wybory,177jq7t,-0.358143,negatywny
2,I obligatoryjnie lepiej jest wsiąść do autobus...,2023-10-13,hermiona52,20,238,0,wybory,176tuaa,-0.551863,negatywny
3,Podajcie swoją propozycje i argumenty zakaz p...,2023-05-18,spidercia,235,32,0,wybory,13l4u1q,-0.737944,negatywny
4,Równo za dwa miesiące wybory Warto wziąć w nic...,2023-08-15,Zealousideal_Life206,89,133,0,wybory,15s31pi,-0.094510,negatywny


In [44]:
# Sumowanie i zliczanie postów dla kategorii sentymentu i tematów
grouped_by_sentiment = wybory_df.groupby(['topic', 'sentiment']).agg({
    'num_comments': 'sum',
    'score': 'sum',
    'downs': 'sum',
    'body': 'count'
}).reset_index()

grouped_by_sentiment = grouped_by_sentiment.rename(columns={'body': 'post_count'})


In [45]:
grouped_by_sentiment

,topic,sentiment,num_comments,score,downs,post_count
0,koalicja,negatywny,667,11066,0,302
1,koalicja,neutralny,310,2640,0,82
2,koalicja,pozytywny,2034,2852,0,85
3,konfederacja,negatywny,3302,21898,0,596
4,konfederacja,neutralny,694,5613,0,132
5,konfederacja,pozytywny,716,5628,0,144
6,pis,negatywny,5847,58084,0,833
7,pis,neutralny,782,10284,0,161
8,pis,pozytywny,553,8619,0,169
9,referendum,negatywny,2681,24264,0,580


In [46]:
fig = px.bar(grouped_by_sentiment, x='topic', y='num_comments',
             color='sentiment', width=800, height=500,
             title='Liczba komentarzy w tematach i sentymentach')
fig.show()

In [47]:
fig = px.bar(grouped_by_sentiment, x='topic', y='score',
             color='sentiment', width=800, height=500,
             title='"score" w tematach i sentymentach')
fig.show()

In [48]:
fig = px.bar(grouped_by_sentiment, x='sentiment', y='num_comments',
             color='topic', width=800, height=500,
             title='Liczba komentarzy do postów o danym wydźwięku')
fig.show()

In [49]:
fig = px.bar(grouped_by_sentiment, x='sentiment', y='score',
             color='topic', width=800, height=500,
             title='"score" dla postów o danym wydźwięku')
fig.show()

In [50]:
wybory_reddit_sentiment = wybory_df
wybory_df.to_csv('wybory_reddit_sentiment.csv', index=False) #zapis

#Modelowanie tematyczne

In [ ]:
wybory_df = pd.read_csv('wybory_reddit_lda.csv')

In [ ]:
!python -m spacy download pl_core_news_sm

In [52]:
import spacy
from spacy.lang.pl.examples import sentences

nlp = spacy.load("pl_core_news_sm")
doc = nlp("jestem na wybory rodacy!")
print(doc.text)
for token in doc:
    print(token.lemma_)

jestem na wybory rodacy!
być
na
wybory
rodaca
!


In [53]:
with open('polish.stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words_pl = set(file.read().splitlines())

In [54]:
texts = wybory_df['body']
processed_texts = []

for doc in nlp.pipe(texts):
    tokens = [token.lemma_.lower() for token in doc]
    tokens_without_stopwords = [token for token in tokens if token not in stop_words_pl]
    processed_texts.append(tokens_without_stopwords)

In [70]:
text_list = [" ".join(lemmatized_words) for lemmatized_words in processed_texts]

##Próba BERTopic

In [59]:
!pip install bertopic
from bertopic import BERTopic

In [84]:
model = BERTopic(nr_topics=3, language="polish")
model

In [85]:
topics, _ = model.fit_transform(text_list)

In [92]:
topic_words = model.get_topics()
topic_words

{-1: [('mieć', 0.05381967779364638),
  ('pis', 0.04696093721533065),
  ('być', 0.03443798082149197),
  ('partia', 0.03189564300161884),
  ('wybory', 0.03185908019333122),
  ('móc', 0.030048024966782653),
  ('chcieć', 0.023920417467480047),
  ('człowiek', 0.023309768787691754),
  ('rok', 0.02130054700075102),
  ('głosować', 0.019963743639324613)],
 0: [('mieć', 0.051946271590236896),
  ('być', 0.03410417911371799),
  ('pis', 0.03245215367377537),
  ('nowy', 0.032255069218999684),
  ('móc', 0.027478845734977),
  ('partia', 0.02697228291284876),
  ('referendum', 0.02696987305682333),
  ('konfederacja', 0.02498260879912005),
  ('rok', 0.024043613115864552),
  ('polska', 0.02381701595582044)],
 1: [('format', 0.32890736705937174),
  ('webps', 0.326876445547862),
  ('width', 0.326876445547862),
  ('pjpgauto', 0.20034464928950135),
  ('pngauto', 0.16312773775985281),
  ('giphy', 0.10432164414562763),
  ('gif', 0.10432164414562763),
  ('xb', 0.10219959540009112),
  ('view', 0.09354096357706783

In [90]:
wybory_df['topics_bert'] = topics

In [94]:
wybory_df.groupby('topics_bert').count()

,body,created,author,num_comments,score,downs,topic,id,sentiment_score,sentiment
topics_bert,,,,,,,,,,
-1,1886,1886,1886,1886,1886,1886,1886,1886,1886,1886
0,2270,2270,2270,2270,2270,2270,2270,2270,2270,2270
1,141,141,141,141,141,141,141,141,141,141


##LDA

In [ ]:
wybory_df['body'][1]

'Pytanie jak w tytule bez oceniania odpowiedzi Dlaczego nie pójdziecie na wybory Czy coś by Was przekonało Czy w poprzednich wyborach braliście udział'

In [109]:
processed_texts[1]

['mieć',
 'pytanie',
 'pierwszy',
 'udział',
 'wybory',
 'siebie',
 'powiązać być',
 'głosywać',
 'poszczególny',
 'kandydat',
 'sejm',
 'senat',
 'partia',
 ' ',
 'dobry',
 'sprawdzić',
 'odpowiadać',
 'pogląd',
 'robiłem',
 'różny',
 'latarnik',
 'wyborczy',
 'kandydat',
 'wypełnić',
 'sprawdzać',
 'postulat',
 ' ',
 'osoba',
 'wchodzić',
 'gra',
 'jutro',
 'wyrobić']

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

min_topics = 2
max_topics = 10
perplexity_scores = []

for n_topics in range(min_topics, max_topics + 1):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([" ".join(text) for text in processed_texts])
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(X)

    perplexity = lda.perplexity(X) #dla zrobieni rankingu najlepszej l_tematów
    perplexity_scores.append(perplexity)

    top_words = []
    for topic_idx, topic in enumerate(lda.components_):
        top_indices = topic.argsort()[-10:][::-1]
        top_words.append([vectorizer.get_feature_names_out()[i] for i in top_indices])
        print(f"Topic #{topic_idx + 1}: {', '.join(top_words[-1])}")

optimal_n_topics = min_topics + perplexity_scores.index(min(perplexity_scores))

print(f"Optimal number of topics: {optimal_n_topics}")

Topic #1: mieć, pis, być, partia, móc, człowiek, wybory, chcieć, by, rok
Topic #2: nowy, referendum, karta, głos, mandat, wynik, komitet, okręg, wyborczy, polska
Topic #1: mieć, pis, być, partia, móc, człowiek, wybory, lewica, chcieć, rok
Topic #2: mieć, referendum, być, pis, karta, wybory, chcieć, głosować, móc, człowiek
Topic #3: nowy, mandat, pis, głos, komitet, okręg, mieć, wynik, polska, partia
Topic #1: mieć, być, lewica, partia, pis, chcieć, konfederacja, polska, rok, podatek
Topic #2: referendum, karta, być, mieć, głosować, chcieć, pytanie, komisja, móc, wybory
Topic #3: nowy, mandat, komitet, okręg, głos, wynik, wyborczyć, kw, procent, pp
Topic #4: mieć, pis, móc, być, partia, wybory, człowiek, chcieć, by, rok
Topic #1: mieć, być, lewica, partia, mówić, podatek, pis, konfederacja, rok, móc
Topic #2: karta, referendum, być, mieć, głosować, link, format, width, webps, chcieć
Topic #3: nowy, mandat, komitet, okręg, wynik, głos, kw, wyborczyć, procent, pp
Topic #4: mieć, pis, part

In [96]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([" ".join(text) for text in processed_texts])

n_topics = 3  # Ilość tematów
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

for topic_idx, topic in enumerate(lda.components_):
    top_indices = topic.argsort()[-10:][::-1] #najważniejsze słowa
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_indices]
    print(f"Temat #{topic_idx + 1}: {', '.join(top_words)}")

Temat #1: mieć, pis, być, partia, móc, człowiek, wybory, lewica, chcieć, rok
Temat #2: mieć, referendum, być, pis, karta, wybory, chcieć, głosować, móc, człowiek
Temat #3: nowy, mandat, pis, głos, komitet, okręg, mieć, wynik, polska, partia


In [97]:
wybory_df['Processed text'] = processed_texts

In [98]:
assigned_topics = []

for text_bow in X:
    topic_distribution = lda.transform(text_bow)
    assigned_topic = topic_distribution.argmax()
    assigned_topics.append(assigned_topic)

In [99]:
wybory_df['LDA_topic'] = assigned_topics

In [100]:
wybory_df.head()

,body,created,author,num_comments,score,downs,topic,id,sentiment_score,sentiment,topics_bert,Processed text,LDA_topic
0,W krótce wybory a ja sobie zdałam sprawę że ni...,2023-08-31,krowajestes,101,48,0,wybory,1668zix,-0.576105,negatywny,0,"[krótka, wybory, siebie, zdać być, sprawa, mie...",0
1,Mam takie pytanie Będzie to mój pierwszy udzia...,2023-10-14,DDDOMIK,34,0,0,wybory,177jq7t,-0.358143,negatywny,0,"[mieć, pytanie, pierwszy, udział, wybory, sieb...",1
2,I obligatoryjnie lepiej jest wsiąść do autobus...,2023-10-13,hermiona52,20,238,0,wybory,176tuaa,-0.551863,negatywny,0,"[obligatoryjnie, wsiąść, autobus, zabrać, choc...",0
3,Podajcie swoją propozycje i argumenty zakaz p...,2023-05-18,spidercia,235,32,0,wybory,13l4u1q,-0.737944,negatywny,-1,"[podajcie, swój, propozycja, argument, , zaka...",1
4,Równo za dwa miesiące wybory Warto wziąć w nic...,2023-08-15,Zealousideal_Life206,89,133,0,wybory,15s31pi,-0.094510,neutralny,-1,"[równo, miesiąc, wybory, warto, wziąć, udział,...",0


In [101]:
def topic_naming(topic_num):
  if topic_num == 0:
    return "Partie"
  elif topic_num == 1:
    return "Czyny PiSu"
  else:
    return "Mechanizm wyborów"

In [102]:
wybory_df['topic_name'] = wybory_df['LDA_topic'].apply(topic_naming)

In [103]:
fig = px.histogram(wybory_df, x='topic_name', width=600, height=400,
             title='Rozkład tematów w postach')
fig.show()

In [104]:
# Wybór popularnych i nie wpisów
df_popular = wybory_df.sort_values(by='num_comments', ascending=False).head(100)
df_unpopular = wybory_df.sort_values(by='num_comments', ascending=True).head(100)

In [105]:
fig = px.bar(df_popular, x='topic_name', color='topic', width=600, height=400,
             title='Rozkład tematów w najpopularniejszych postach')
fig.show()

In [106]:
fig = px.bar(df_unpopular, x='topic_name', color='topic', width=600, height=400,
             title='Rozkład tematów w najmniej popularnych postach')
fig.show()

In [107]:
wybory_lda_grouped = wybory_df.groupby('topic_name')['num_comments'].sum().reset_index(name='count')
wybory_lda_grouped

,topic_name,count
0,Czyny PiSu,8869
1,Mechanizm wyborów,3817
2,Partie,12571


In [108]:
fig = px.bar(wybory_lda_grouped, x='topic_name', y='count', width=600, height=400,
             title='Popularność postów w podziale na grupy tematyczne')
fig.show()

In [110]:
wybory_df.to_csv('wybory_reddit_final.csv', index=False) #zapis